<a href="https://colab.research.google.com/github/ShoAnn/Legal_Document_IR/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Load dataset**



In [1]:
import pandas as pd
import gdown

In [2]:
dataset = 'perdata_200.json'
gdown.download(f'https://drive.google.com/file/d/1jJwJAP2kkR_qiGIcJFDVpErY9ee7qmuS/view?usp=drive_link', dataset, quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1jJwJAP2kkR_qiGIcJFDVpErY9ee7qmuS
To: /content/perdata_200.json
100%|██████████| 31.5M/31.5M [00:00<00:00, 115MB/s]


'perdata_200.json'

In [29]:
#annotated dataset putusan pidana scraped from https://putusan3.mahkamahagung.go.id/
df = pd.read_json(dataset)
print(df['text'][152])
print(df['text'][159])
print(df['text'][160])

['oo', 'edod', 'putusan', 'oo']
['oo', 'edod', 'putusan', 'oo']
['oo', 'edod', 'putusan', 'oo']


In [4]:
data = []
for i, row in df.iterrows():
  new_text = []
  new_label = []
  for j in range(len(row['label'])-1):
    if row['label'][j] != 'O' :
      new_text.append(row['text'][j])
      new_label.append(row['label'][j])
  data.append([new_text, new_label])

In [20]:
def group_entities(tokens, labels):
    entity_groups = {}
    current_entity = None
    current_group = []

    for token, label in zip(tokens, labels):
        if label.startswith('B_'):
            # Start of a new entity
            if current_group:
                entity_groups[current_entity] = current_group
            current_entity = label[2:]  # Extract entity type
            current_group = [token]
        elif label.startswith('I_'):
            # Continue current entity
            if current_entity:
                current_group.append(token)
        else:
            # Outside of any entity
            pass

    # Add the last entity group
    if current_group:
        entity_groups[current_entity] = current_group

    return entity_groups

In [26]:
grouped_data = []
for row in data:
  group = group_entities(row[0], row[1])
  grouped_data.append(group)

for x in range(len(grouped_data)):
    if grouped_data[x] == {}:
      print(x)
    else:
      print(grouped_data[x])
print(len(grouped_data))

{'VERN': ['698', '/', 'pdt', '.', 'g', '/', '2021', '/', 'pn', 'jkt', '.', 'pst'], 'PLAN': ['amirul', 'muchtar', ',', 'se'], 'DENN': ['mahkamah', 'partai', 'nasdem', ','], 'LSWR': ['menyatakan', 'pengadilan', 'negeri', 'tidak', 'berwenang', 'mengadili', 'perkara', 'ini'], 'JUDP': ['suparman', ','], 'JUG': ['bambang', 'sucipto', ','], 'TIMV': ['09', 'agustus', '2022'], 'REGI': ['friska', 'silitonga', ',']}
{'PLAN': ['henri', 'kusuma', ','], 'DENN': ['prasetyo'], 'PLAO': ['.', 'dewan', 'pimpinan', 'pusat', 'kongres', 'advokat', 'indonesia'], 'TIMV': ['04', 'agustus', '2022'], 'LSWR': ['dicabut'], 'JUDP': ['bambang', 'sucipto'], 'JUG': ['dariyanto', ','], 'REGI': ['tambat', 'akbar', ',']}
{'PLAN': ['chris', 'inderayanto'], 'DENN': ['pt', 'bank', 'negara', 'indonesia', '(', 'persero', ')', 'tbk', '.'], 'PLAO': ['pemerintah', 'republik', 'indonesia', 'cq', '.', 'kementerian', 'keuangan', 'republik', 'indonesia', 'cq', '.', 'direktorat', 'jendral', 'kekayaan', 'negara', 'cq', '.', 'kantor', 

In [27]:
data[159]

[[], []]

In [6]:
new_df = pd.DataFrame(data, columns=['text', 'label'])
new_df

,text,label
0,"[698, /, pdt, ., g, /, 2021, /, pn, jkt, ., ps...","[B_VERN, I_VERN, I_VERN, I_VERN, I_VERN, I_VER..."
1,"[henri, kusuma, ,, zeesha, fatma, defaga, ,, s...","[B_PLAN, I_PLAN, I_PLAN, B_DENN, I_DENN, I_DEN..."
2,"[chris, inderayanto, ,, pt, ., bank, negara, i...","[B_PLAN, I_PLAN, I_PLAN, B_DENN, I_DENN, I_DEN..."
3,"[h, ., rayan, nurachman, islamiati, lediana, ,...","[B_PLAN, I_PLAN, I_PLAN, I_PLAN, B_DENN, I_DEN..."
4,"[818, /, pdt, ., g, /, 2021, /, pn, jkt, ., ps...","[B_VERN, I_VERN, I_VERN, I_VERN, I_VERN, I_VER..."
...,...,...
195,"[787, /, pdt, ., g, /, 2023, /, pn, ., sby, .,...","[B_VERN, I_VERN, I_VERN, I_VERN, I_VERN, I_VER..."
196,"[pt, ., granting, jaya, pt, ., granting, jaya,...","[B_PLAN, I_PLAN, I_PLAN, I_PLAN, B_PLAN, I_PLA..."
197,"[anugrah, yudo, witjaksono, ,, pt, ., prima, m...","[B_PLAN, I_PLAN, I_PLAN, I_PLAN, B_DENN, I_DEN..."
198,"[680, /, pdt, ., g, /, 2023, /, pn, sby, ., 68...","[B_VERN, I_VERN, I_VERN, I_VERN, I_VERN, I_VER..."


In [ ]:


grouped_entities = group_entities()

>## *Formatting*

In [ ]:
#fill empty/none value with ' ' for text, and 'O' for tags
df['text'] = df.apply(lambda row: [x if x is not None else ' ' for x in row['text']], axis=1)
df['text-tags'] = df.apply(lambda row: [x if x is not None else 'O' for x in row['text-tags']], axis=1)

In [ ]:
new_df= pd.DataFrame([])

In [ ]:
new_df['text'] = df["text"].apply(lambda x: " ".join(x))
new_df['labels'] = df["text-tags"].apply(lambda x: " ".join(x))
new_df

,text,labels
0,PUTUSAN . NOMOR : 1974 / Pid . Sus / 2012 / PN...,O O O O B_VERN I_VERN I_VERN I_VERN I_VERN I_V...
1,PUTUSAN Nomor 1368 / Pid . B / 2018 / PN Jkt ....,O O B_VERN I_VERN I_VERN I_VERN I_VERN I_VERN ...
2,PUTUSAN Nomor 985 / Pid . Sus / 2017 / PN JKT ...,O O B_VERN I_VERN I_VERN I_VERN I_VERN I_VERN ...
3,PUTUSAN No . 924 / Pid . B / 2014 / PN . Jkt ....,O O O B_VERN I_VERN I_VERN I_VERN I_VERN I_VER...
4,PUTUSAN Nomor 68 / Pid . B / 2016 / PN . JKT ....,O O B_VERN I_VERN I_VERN I_VERN I_VERN I_VERN ...
...,...,...
1095,1 putusan no . 21 / pid . b / 2012 / pn . tmg ...,O O O O B_VERN I_VERN I_VERN I_VERN I_VERN I_V...
1096,dir1ektori putusan mahkamah agung republik ind...,O O O O O O O O O B_VERN I_VERN I_VERN I_VERN ...
1097,putusan nomor : 376 / pid . b / 2012 / pn . kd...,O O O B_VERN I_VERN I_VERN I_VERN I_VERN I_VER...
1098,putusan nomor : 79 / pid . sus / 2015 / pn . b...,O O O B_VERN I_VERN I_VERN I_VERN I_VERN I_VER...


In [ ]:
test_df = pd.DataFrame([])
test_df['document'] = new_df["text"]
test_df.head()

,document
0,PUTUSAN . NOMOR : 1974 / Pid . Sus / 2012 / PN...
1,PUTUSAN Nomor 1368 / Pid . B / 2018 / PN Jkt ....
2,PUTUSAN Nomor 985 / Pid . Sus / 2017 / PN JKT ...
3,PUTUSAN No . 924 / Pid . B / 2014 / PN . Jkt ....
4,PUTUSAN Nomor 68 / Pid . B / 2016 / PN . JKT ....


>## *Save to csv*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
test_df.to_csv("/content/drive/MyDrive/Legal_IR/pidana_1100_textonly.csv")

# **Feature extraction**

In [ ]:
from transformers import BertTokenizerFast, BertForTokenClassification # model tokenizer
from tqdm import tqdm #progress bar
import requests
import torch
import re

>## Load model





>>### Import Base Bert model

In [ ]:
class BertModel(torch.nn.Module):

    def __init__(self, num_labels):

        super(BertModel, self).__init__()
        self.bert = BertForTokenClassification.from_pretrained('indolem/indobert-base-uncased', num_labels=num_labels)

    def forward(self, input_id, mask, label):

        output = self.bert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)

        return output

In [ ]:
# Entities to be predicted
ids_to_labels = {0: 'B_ADVO', 1: 'B_ARTV', 2: 'B_CRIA', 3: 'B_DEFN', 4: 'B_JUDG', 5: 'B_JUDP', 6: 'B_PENA', 7: 'B_PROS', 8: 'B_PUNI', 9: 'B_REGI', 10: 'B_TIMV', 11: 'B_VERN', 12: 'I_ADVO', 13: 'I_ARTV', 14: 'I_CRIA', 15: 'I_DEFN', 16: 'I_JUDG', 17: 'I_JUDP', 18: 'I_PENA', 19: 'I_PROS', 20: 'I_PUNI', 21: 'I_REGI', 22: 'I_TIMV', 23: 'I_VERN', 24: 'O', 25: 'nan'}

# Specify number of neuron in output layer to match the number of entities
model = BertModel(len(ids_to_labels))

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at indolem/indobert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


>>### Import Fine-Tuned bert for NER

In [ ]:
gdown.download(f'https://drive.google.com/file/d/1uFwimITnqeENoJiLoRuOsf_YpLpOIM5h/view?usp=sharing', 'IndoBERT-LegalNER.pth', quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1uFwimITnqeENoJiLoRuOsf_YpLpOIM5h
To: /content/IndoBERT-LegalNER.pth
100%|██████████| 440M/440M [00:03<00:00, 123MB/s]


'IndoBERT-LegalNER.pth'

In [ ]:
#Load Fine-tuned model to base model
model_weights = torch.load('IndoBERT-LegalNER.pth', map_location=torch.device('cpu'))
model.load_state_dict(model_weights)
model.eval()

BertModel(
  (bert): BertForTokenClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(31923, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=Tru

>## Prediction

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('indolem/indobert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/234k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
label_all_tokens = True
def align_word_ids(texts):
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(1)
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(1 if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx

    return label_ids

In [ ]:
new_data = test_df['document'][0]
new_data

'PUTUSAN . NOMOR : 1974 / Pid . Sus / 2012 / PN . JKT . BAR . DEMI KEADILAN BERDASARKAN KETUHANAN YANG MAHA ESA . Pengadilan Negeri Jakarta Barat yang memeriksa dan mengadili perkara - perkara pidana dalam peradilan tingkat pertama dengan acara biasa telah menjatuhkan putusan sebagai berikut atas nama terdakwa : AGNES TRI AHADI Als ANAS , lahir di Jakarta , umur / tanggal , 30 tahun / 8 Agustus 1982 , jenis kelamin laki - laki , kebangsaan Indonesia , tempat tinggal Jl . Kamp . Bali Gg . II . No . 5 Kel . Kampung Bali Kec . Tanah Abang Jakarta Pusat Terdakwa ditahan sejak tanggal 27 Juli 2012 sampai sekarang ; Terdakwa menolak didampingi oleh Penasehat Hukum ; Pengadilan Negeri tersebut ; Setelah membaca berkas perkara ; Setelah mendengar keterangan saksi - saksi dan terdakwa dipersidangan ; Setelah mendengar tuntutan pidana dari Jaksa Penuntut Umum tertanggal 9 Januari 2013 , yang pada pokoknya agar Pengadilan Negeri Jakarta Barat berkenan memutuskan sebagai berikut : MENUNTUT : 1 Men

In [ ]:
new_data_tokenized = tokenizer(new_data, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")
input_ids_data = new_data_tokenized['input_ids'].to('cpu')
mask_data = new_data_tokenized['attention_mask'].to('cpu')

print(new_data_tokenized.word_ids())

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 142, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 205, 206, 207, 208, 209, 210, 211, 212, 212, 213, 

In [ ]:
input_ids_conv = tokenizer.convert_ids_to_tokens(new_data_tokenized['input_ids'][0])
print(input_ids_conv)

data_token = [word for word in input_ids_conv if word not in ['[CLS]', '[SEP]', '[PAD]']]
print(data_token)

['[CLS]', 'putusan', '.', 'nomor', ':', '1974', '/', 'pid', '.', 'sus', '/', '2012', '/', 'pn', '.', 'jkt', '.', 'bar', '.', 'demi', 'keadilan', 'berdasarkan', 'ketuhanan', 'yang', 'maha', 'esa', '.', 'pengadilan', 'negeri', 'jakarta', 'barat', 'yang', 'memeriksa', 'dan', 'mengadili', 'perkara', '-', 'perkara', 'pidana', 'dalam', 'peradilan', 'tingkat', 'pertama', 'dengan', 'acara', 'biasa', 'telah', 'menjatuhkan', 'putusan', 'sebagai', 'berikut', 'atas', 'nama', 'terdakwa', ':', 'agnes', 'tri', 'ahad', '##i', 'als', 'anas', ',', 'lahir', 'di', 'jakarta', ',', 'umur', '/', 'tanggal', ',', '30', 'tahun', '/', '8', 'agustus', '1982', ',', 'jenis', 'kelamin', 'laki', '-', 'laki', ',', 'kebangsaan', 'indonesia', ',', 'tempat', 'tinggal', 'jl', '.', 'kamp', '.', 'bali', 'g', '##g', '.', 'ii', '.', 'no', '.', '5', 'kel', '.', 'kampung', 'bali', 'kec', '.', 'tanah', 'abang', 'jakarta', 'pusat', 'terdakwa', 'ditahan', 'sejak', 'tanggal', '27', 'juli', '2012', 'sampai', 'sekarang', ';', 'terdak

In [ ]:
logits_data = model(input_ids_data, mask_data, None)
label_ids_data = torch.Tensor(align_word_ids(new_data)).unsqueeze(0).to('cpu')
logits_clean_data = logits_data[0][label_ids_data != -100]


predictions_data = logits_clean_data.argmax(dim=1).tolist()
prediction_label_data = [ids_to_labels[i] for i in predictions_data]


print(new_data)
print(prediction_label_data)

PUTUSAN . NOMOR : 1974 / Pid . Sus / 2012 / PN . JKT . BAR . DEMI KEADILAN BERDASARKAN KETUHANAN YANG MAHA ESA . Pengadilan Negeri Jakarta Barat yang memeriksa dan mengadili perkara - perkara pidana dalam peradilan tingkat pertama dengan acara biasa telah menjatuhkan putusan sebagai berikut atas nama terdakwa : AGNES TRI AHADI Als ANAS , lahir di Jakarta , umur / tanggal , 30 tahun / 8 Agustus 1982 , jenis kelamin laki - laki , kebangsaan Indonesia , tempat tinggal Jl . Kamp . Bali Gg . II . No . 5 Kel . Kampung Bali Kec . Tanah Abang Jakarta Pusat Terdakwa ditahan sejak tanggal 27 Juli 2012 sampai sekarang ; Terdakwa menolak didampingi oleh Penasehat Hukum ; Pengadilan Negeri tersebut ; Setelah membaca berkas perkara ; Setelah mendengar keterangan saksi - saksi dan terdakwa dipersidangan ; Setelah mendengar tuntutan pidana dari Jaksa Penuntut Umum tertanggal 9 Januari 2013 , yang pada pokoknya agar Pengadilan Negeri Jakarta Barat berkenan memutuskan sebagai berikut : MENUNTUT : 1 Meny

In [ ]:
def convertTotext(data_token, prediction_label):
  prev_tag = 'O'
  result = {}
  temp = ''
  for i, word in enumerate(data_token):
    if prediction_label[i] != 'O':
      if prev_tag == 'O' and temp != '':
        temp = ''

      if '##' in word:
        temp += word.replace('##', '')

      else:
        temp +=  ' ' + word
    else:
      if temp != "":
        result[prev_tag.replace("I", "B")] = temp.strip()
      temp = ""

    prev_tag = prediction_label[i]

  return result

In [ ]:
convertTotext(data_token, prediction_label_data)

{'B_VERN': '1974 / pid . sus / 2012 / pn . jkt . bar .',
 'B_ARTV': 'pasal 112 ayat ( 1 ) uu ri no . 35 tahun 2009 , tentang',
 'B_PENA': 'jara'}

>>## Tokenize